In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import torch
import numpy as np
from copy import deepcopy
from sklearn.metrics import mean_squared_error

# Benchmarks

## 1. No adjusts

In [ ]:
# electricity load dataset (persistence: loop)
# pth = "results/mfred/example_benchmarks-20230831-041824"

# wind power dataset (persistence: naive)
pth = "results/nrel/example_benchmarks-20230817-231811"
# pth = "results/mfred/example_benchmarks-20230814-210128"

# NL with one large Laplace decoder (failure)
# pth = "results/mfred/example_benchmarks-20231012-175829"

# model_names = ['Neural Laplace', 'LSTM', 'MLP']
model_names = ['Neural Laplace', 'LSTM', 'MLP', 'Persistence']
# model_names = ['Neural Laplace']
fcst_features = [0]
all_seed_result = []
all_seed_preds = []
for seed in range(20):
    with open(f"{pth}-{seed}.pkl", "rb") as f:
        all_result = pickle.load(f)
    test_result = {name: {} for name in model_names}
    test_preds_trajs = {name: {} for name in model_names}
    for avg_terms in all_result:
        result_avg = all_result[avg_terms]
        train_mean = result_avg['train_mean'][fcst_features]
        train_std = result_avg['train_std'][fcst_features]
        num_avg_terms = int(avg_terms.split("_")[-1])
        for name in model_names:
            model_results = result_avg[name]

            test_preds = model_results["test_preds"] * train_std.cpu().numpy(
            ) + train_mean.cpu().numpy()
            test_trajs = model_results["test_trajs"] * train_std.cpu().numpy(
            ) + train_mean.cpu().numpy()
            pred_timesteps = test_trajs.shape[1]
            # if num_avg_terms == 1:
            #     fig, ax = plt.subplots()
            #     ax.plot(test_trajs[300,:,0], label="real")
            #     ax.plot(test_preds[300,:,0], label="fcst")
            #     ax.legend()
            #     ax.set(xlabel="Forecasting horizon", ylabel="Load(kW)")
            #     # ax.set_title(name)
            #     fig.savefig("savings/supp_NL.pdf")
            test_result[name][num_avg_terms] = mean_squared_error(
                test_trajs.squeeze(), test_preds.squeeze(), squared=False)
            test_preds_trajs[name][num_avg_terms] = deepcopy(
                (test_preds, test_trajs))
    df_test_result = pd.DataFrame(test_result)
    all_seed_preds.append(test_preds_trajs)
    all_seed_result.append(df_test_result)
all_seed_result = pd.concat(all_seed_result, axis=0)
all_seed_result = all_seed_result.groupby(all_seed_result.index).agg(
    ["mean", "std"])
all_seed_result = all_seed_result.sort_index()
avg_terms_list = all_seed_result.index.tolist()
all_seed_result
# with open(f'savings/bench_20_{pth.split("/")[1]}.pickle', 'wb') as handle:
#     pickle.dump(all_seed_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)

mean consistent error

In [ ]:
max_avg_terms = max(avg_terms_list)
mce_results = {name: [] for name in model_names}
for seed in range(20):
    preds_trajs_dict = all_seed_preds[seed]
    for name in model_names:
        mce = {"5min vs 15min":0, "5min vs 60min":0, "15min vs 60min":0}
        for n, avg_terms in enumerate(avg_terms_list):

            if avg_terms < max_avg_terms:
                # print("current avg_terms", avg_terms)
                rest_avg_terms = avg_terms_list[n + 1:]
                (test_preds, test_trajs) = preds_trajs_dict[name][avg_terms]
                for r_avg_terms in rest_avg_terms:
                    (real_avg_preds,
                     real_avg_trajs) = preds_trajs_dict[name][r_avg_terms]
                    # print(avg_terms, r_avg_terms)
                    # calculate predictions
                    avg_test_preds = np.split(test_preds,
                                              test_preds.shape[1] //
                                              (r_avg_terms // avg_terms),
                                              axis=1)
                    avg_test_preds = np.stack(
                        [j.mean(axis=1) for j in avg_test_preds], axis=1)
                    error = (avg_test_preds - real_avg_preds)**2
                    mce[f"{avg_terms*5}min vs {r_avg_terms*5}min"] += error.mean()
        # mce = pd.DataFrame(mce, index=0)
        # print(mce)
        mce_results[name].append(mce)

all_model_mce = []
for name in model_names:
    
    model_mce = pd.DataFrame(mce_results[name]).mean()
    all_model_mce.append(model_mce)
all_model_mce = pd.concat(all_model_mce, axis=1)
all_model_mce = all_model_mce.T
all_model_mce.index = model_names
all_model_mce = all_model_mce.drop("Persistence", axis=0)
all_model_mce
# mce_results.index = [0 for _ in range(20)]
# mce_results = mce_results.groupby(mce_results.index).agg(
    # ["mean", "std"])
# mce_results.index= ["MCE"]
# mce_results.mean()
# mce_results.to_csv(f"savings/bench_{pth.split('/')[1]}_mce.csv")

## 2. After adjusted

### 2.1 Bottom up

In [ ]:
all_bu_results, all_bu_preds = [], []
for seed in range(20):
    preds_trajs_dict = all_seed_preds[seed]
    bu_results = {name + "-BU": {} for name in model_names}
    bu_preds = {name + "-BU": {} for name in model_names}
    for name in model_names:
        for avg_terms in avg_terms_list:
            (test_preds, test_trajs) = preds_trajs_dict[name][1]
            # Test trajs are consistent
            avg_test_trajs = np.split(test_trajs,
                                      test_trajs.shape[1] // avg_terms,
                                      axis=1)
            avg_test_trajs = np.stack([j.mean(axis=1) for j in avg_test_trajs],
                                      axis=1)

            # calculate predictions
            avg_test_preds = np.split(test_preds,
                                      test_preds.shape[1] // avg_terms,
                                      axis=1)
            avg_test_preds = np.stack([j.mean(axis=1) for j in avg_test_preds],
                                      axis=1)
            bu_results[name + "-BU"][avg_terms] = mean_squared_error(
                avg_test_trajs.flatten(),
                avg_test_preds.flatten(),
                squared=False)
            bu_preds[name + "-BU"][avg_terms] = (avg_test_preds,avg_test_trajs
                                                 )

            # if avg_terms == 3:
            # fig, ax = plt.subplots()
            # ax.plot(avg_test_trajs[0,:,0])
            # ax.plot(avg_test_preds[0,:,0])

    bu_results = pd.DataFrame(bu_results)
    all_bu_results.append(bu_results)
    all_bu_preds.append(bu_preds)
all_bu_results = pd.concat(all_bu_results, axis=0)
all_bu_results = all_bu_results.groupby(all_bu_results.index).agg(
    ["mean", "std"])
all_bu_results = all_bu_results.sort_index()
all_bu_results
# with open(f'savings/bench_bu_20_{pth.split("/")[1]}.pickle', 'wb') as handle:
#     pickle.dump(all_bu_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
# all_bu_results.to_csv("savings/benchmark_bu.csv")


### 2.2 Optimized

Athanasopoulos, George, et al. "Forecasting with temporal hierarchies." European Journal of Operational Research 262.1 (2017): 60-74.

In [ ]:
from scipy.linalg import block_diag

S = [
    block_diag(*[[1.0 / s for _ in range(s)]
                 for _ in range(pred_timesteps // s)]) for s in avg_terms_list
]
S.reverse()
S = np.concatenate(S)
S = S @ np.linalg.inv(S.T @ S) @ S.T

all_opt_results, all_opt_preds = [], []
for seed in range(20):
    preds_trajs_dict = all_seed_preds[seed]
    opt_results = {name + "-OPT": {} for name in model_names}
    opt_preds = {name + "-OPT": {} for name in model_names}
    for name in model_names:
        preds_vec, trajs_vec, pred_steps = [], [], []
        for avg_terms in avg_terms_list[::-1]:
            (test_preds, test_trajs) = preds_trajs_dict[name][avg_terms]
            preds_vec.append(test_preds.copy())
            trajs_vec.append(test_trajs.copy())
        preds_vec = np.concatenate(preds_vec, axis=1)
        # preds_vec = preds_vec.transpose(1, 0, 2)
        adjusted_preds = np.tensordot(S, preds_vec, axes=[1, 1])
        start = 0
        for i, trajs in enumerate(trajs_vec):
            steps = trajs.shape[1]
            opt_results[name +
                        "-OPT"][avg_terms_list[::-1][i]] = mean_squared_error(
                            trajs.flatten(),
                            adjusted_preds[start:start + steps,
                                           ...].transpose(1, 0, 2).flatten(),
                            squared=False)
            opt_preds[name + "-OPT"][avg_terms_list[::-1][i]] = (
                adjusted_preds[start:start + steps, ...].transpose(1, 0,
                                                                   2), trajs)
            # if i == 0:
            #     fig, ax = plt.subplots()
            #     ax.plot(trajs[0].flatten())
            #     ax.plot(adjusted_preds[start:start + steps, 0, :].flatten())
            #     print(adjusted_preds[start:start + steps, 0, :].flatten())
            start += steps

    opt_results = pd.DataFrame(opt_results)
    all_opt_results.append(opt_results)
    all_opt_preds.append(opt_preds)

all_opt_results = pd.concat(all_opt_results, axis=0)
all_opt_results = all_opt_results.groupby(all_opt_results.index).agg(
    ["mean", "std"])
all_opt_results = all_opt_results.sort_index()
# all_opt_results.to_csv("savings/benchmark_opt.csv")
# with open(f'savings/bench_opt_20_{pth.split("/")[1]}.pickle', 'wb') as handle:
#     pickle.dump(all_opt_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
all_opt_results

# Hierarchical NL

## 1. No adjusts

In [ ]:
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd


# Load
pth = "results/mfred/example-20230815-052223"

# Wind
# pth = "results/nrel/example-20230819-070441"


all_seed_result = []
all_seed_preds = []
fcst_features = [0]
for seed in range(20):
    with open(f"{pth}-{seed}.pkl", "rb") as f:
        result = pickle.load(f)
        avg_terms_list = result["model_hyperparams"]["avg_terms_list"]
        # print(result["model_hyperparams"]["encoder"])
        train_mean = result["train_mean"][fcst_features].detach().cpu().numpy()
        train_std = result["train_std"][fcst_features].detach().cpu().numpy()

        model = result["Hierarchical NL"]
        print(model["train_losses"][:10])
        # avg_terms_list = result["avg_terms_list"]
        # print(result["train_mean"])
        # print(result["train_std"])
        test_label = model["test_trajs"]
        test_preds = model["test_preds"]
        test_label = test_label
        test_label = test_label * train_std + train_mean
        pred_timesteps = test_label.shape[1]

    avg_test_label, avg_test_preds, rmse = {}, {}, {}
    for i, avg_terms in enumerate(avg_terms_list):
        temp = np.split(test_label, test_label.shape[1] // avg_terms, axis=1)
        temp = np.stack([j.mean(axis=1) for j in temp], axis=1)
        avg_test_label[avg_terms] = temp
        # avg_test_preds[avg_terms] = (test_preds[i].detach().cpu().numpy(
        # ), temp)
        avg_test_preds[avg_terms] = (
            test_preds[i].detach().cpu().numpy() * train_std + train_mean,
            temp)

        # print(avg_test_preds[avg_terms][0].flatten())
        rmse[avg_terms] = [
            mean_squared_error(temp.squeeze(),
                               avg_test_preds[avg_terms][0].squeeze(),
                               squared=False)
        ]
        # fig, ax = plt.subplots()
        # ax.plot(temp[3,:,0])
        # ax.plot(avg_test_preds[avg_terms][0][3,:,0])
    rmse = pd.DataFrame(rmse).transpose()
    all_seed_result.append(rmse)
    all_seed_preds.append(avg_test_preds)
all_seed_result = pd.concat(all_seed_result, axis=0)
all_seed_result = all_seed_result.groupby(all_seed_result.index).agg(
    ["mean", "std"])
avg_terms_list = all_seed_result.index.tolist()
all_seed_result = all_seed_result.rename(columns={0: "Hierarchical NL"})
all_seed_result

# with open(f'savings/proposed_20_{pth.split("/")[1]}.pickle', 'wb') as handle:
#     pickle.dump(all_seed_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)

mean consistence error

In [ ]:
max_avg_terms = max(avg_terms_list)
mce_results = {}
for seed in range(20):
    preds_trajs_dict = all_seed_preds[seed]
    mce = {"5min vs 15min":0, "5min vs 60min":0, "15min vs 60min":0}
    for n, avg_terms in enumerate(avg_terms_list):

        if avg_terms < max_avg_terms:
            # print("current avg_terms", avg_terms)
            rest_avg_terms = avg_terms_list[n + 1:]
            test_preds, _ = preds_trajs_dict[avg_terms]
            for r_avg_terms in rest_avg_terms:
                real_avg_preds, _ = preds_trajs_dict[r_avg_terms]

                # print(avg_terms, r_avg_terms)
                # calculate predictions
                avg_test_preds = np.split(test_preds,
                                          test_preds.shape[1] //
                                          (r_avg_terms // avg_terms),
                                          axis=1)
                avg_test_preds = np.stack(
                    [j.mean(axis=1) for j in avg_test_preds], axis=1)
                error = (avg_test_preds - real_avg_preds)**2
                mce[f"{avg_terms*5}min vs {r_avg_terms*5}min"] += error.mean()
    mce_results[seed] = mce
    # mce_results[seed] = [mce]
mce_results = pd.DataFrame(mce_results)
mce_results.index.name = "seed"
mce_results = mce_results.transpose()
print(mce_results)
# mce_results.columns=["Hierarchical NL"]
# mce_results.index = [0 for _ in range(20)]
mce_results = mce_results.mean()
# mce_results.index= ["MCE"]
all_model_mce = pd.concat([all_model_mce.T, mce_results], axis=1)
all_model_mce.rename(columns={0:"HL"}, inplace=True)
all_model_mce
# hnl = pd.concat([all_seed_result, mce_results])
# benchmarks = pd.read_csv("savings/benchmark_raw.csv", index_col=0, header=[0,1])
# benchmarks.index = hnl.index
# all_results = pd.concat([benchmarks, hnl], axis=1)
# print(all_results)
# mce_results.to_csv(f"savings/proposed_{pth.split('/')[1]}_mce.csv")

In [ ]:
all_model_mce = all_model_mce.sort_values(by = "15min vs 60min", axis=1)
all_model_mce = all_model_mce.rename(columns={"Neural Laplace":"NL"})
all_model_mce.to_csv("wind_mce.csv")
colors = plt.cm.BuPu(np.linspace(0.1, 0.5, len(all_model_mce)))
n_rows = len(all_model_mce)

index = np.arange(len(all_model_mce.columns))
bar_width = 0.6

# Initialize the vertical-offset for the stacked bar chart.
y_offset = np.zeros(len(all_model_mce.columns))

fig, ax=plt.subplots()
# Plot bars and create text labels for the table
cell_text = []
for row in range(n_rows):
    ax.bar(all_model_mce.columns, all_model_mce.iloc[row, :], bar_width, bottom=y_offset, color=colors[row], label=all_model_mce.index[row])
    y_offset = y_offset + all_model_mce.iloc[row, :]

ax.legend(fontsize=12)
for i, tick in enumerate(ax.get_xticklabels()):
    tick.set_fontsize(14)

    if i == 0:
        tick.set_fontweight('bold')
        tick.set_fontsize(16)

for i, tick in enumerate(ax.get_yticklabels()):
    tick.set_fontsize(12)
ax.ticklabel_format(style='sci', scilimits=(-1,2), axis='y')
# fig.savefig("savings/mce_wind.pdf",bbox_inches="tight")


In [ ]:
load_mce = pd.read_csv("savings/load_mce.csv", index_col=0)
wind_mce = pd.read_csv("savings/wind_mce.csv", index_col=0)

colors = plt.cm.BuPu(np.linspace(0.1, 0.5, len(wind_mce)))
n_rows = len(wind_mce)

index = np.arange(len(wind_mce.columns))
bar_width = 0.6

# Initialize the vertical-offset for the stacked bar chart.
y_offset_load = np.zeros(len(load_mce.columns))
y_offset_wind = np.zeros(len(wind_mce.columns))

fig, axs=plt.subplots(1,2,figsize=[14,6])
# Plot bars and create text labels for the table
cell_text = []
for row in range(n_rows):
    axs[0].bar(load_mce.columns, load_mce.iloc[row, :], bar_width, bottom=y_offset_load, color=colors[row], label=load_mce.index[row])
    axs[1].bar(wind_mce.columns, wind_mce.iloc[row, :], bar_width, bottom=y_offset_wind, color=colors[row], label=wind_mce.index[row])
    y_offset_load = y_offset_load + load_mce.iloc[row, :]
    y_offset_wind = y_offset_wind + wind_mce.iloc[row, :]

for n in range(2):
    for i, tick in enumerate(axs[n].get_xticklabels()):
        tick.set_fontsize(14)

        if i == 0:
            tick.set_fontweight('bold')
            tick.set_fontsize(16)

    for i, tick in enumerate(axs[n].get_yticklabels()):
        tick.set_fontsize(12)
    axs[n].ticklabel_format(style='sci', scilimits=(-1,2), axis='y')
fig.tight_layout()
axs[0].legend(fontsize="x-large",ncol=3,
    loc='center',
    bbox_to_anchor=(1.05, 1.075),
    fancybox=True,)
# fig.savefig("savings/mce_new.pdf",bbox_inches="tight")


## 2. After adjusted

### 2.1 BU

In [ ]:
all_bu_results = []
for seed in range(20):
    preds_trajs_dict = all_seed_preds[seed]
    bu_results = {}

    for avg_terms in avg_terms_list:
        (test_preds, test_trajs) = preds_trajs_dict[1]
        # Test trajs are consistent
        avg_test_trajs = np.split(test_trajs,
                                    test_trajs.shape[1] // avg_terms,
                                    axis=1)
        avg_test_trajs = np.stack([j.mean(axis=1) for j in avg_test_trajs],
                                    axis=1)

        # calculate predictions
        avg_test_preds = np.split(test_preds,
                                    test_preds.shape[1] // avg_terms,
                                    axis=1)
        avg_test_preds = np.stack([j.mean(axis=1) for j in avg_test_preds],
                                    axis=1)
        bu_results[avg_terms] = [mean_squared_error(
            avg_test_trajs.flatten(),
            avg_test_preds.flatten(),
            squared=False)]
            # if avg_terms == 3:
            # fig, ax = plt.subplots()
            # ax.plot(avg_test_trajs[0,:,0])
            # ax.plot(avg_test_preds[0,:,0])
            
    bu_results = pd.DataFrame(bu_results)
    all_bu_results.append(bu_results)
all_bu_results = pd.concat(all_bu_results, axis=0)
all_bu_results = all_bu_results.groupby(all_bu_results.index).agg(
    ["mean", "std"])
all_bu_results = all_bu_results.sort_index()
all_bu_results

In [ ]:
from scipy.linalg import block_diag

S = [
    block_diag(*[[1.0 / s for _ in range(s)] for _ in range(pred_timesteps // s)])
    for s in avg_terms_list
]
S.reverse()
S = np.concatenate(S)
S = S @ np.linalg.inv(S.T @ S) @ S.T

all_opt_results, all_opt_preds = [], []
avg_terms_list.sort(reverse=True)
for seed in range(20):
    preds_trajs_dict = all_seed_preds[seed]
    opt_results = {}
    preds_vec, trajs_vec, pred_steps = [], [], []
    for avg_terms in avg_terms_list:
        (test_preds, test_trajs) = preds_trajs_dict[avg_terms]
        preds_vec.append(test_preds.copy())
        trajs_vec.append(test_trajs.copy())
    preds_vec = np.concatenate(preds_vec, axis=1)
    # preds_vec = preds_vec.transpose(1, 0, 2)
    adjusted_preds = np.tensordot(S, preds_vec, axes=[1, 1])
    start = 0
    for i, trajs in enumerate(trajs_vec):
        steps = trajs.shape[1]
        opt_results[avg_terms_list[i]] = [mean_squared_error(
            trajs.flatten(),
            adjusted_preds[start:start + steps, ...].transpose(1,0,2).flatten(),
            squared=False)]
        # if i == 0:
        #     fig, ax = plt.subplots()
        #     ax.plot(trajs[0].flatten())
        #     ax.plot(adjusted_preds[start:start + steps, 0, :].flatten())
        #     print(adjusted_preds[start:start + steps, 0, :].flatten())
        start += steps

    opt_results = pd.DataFrame(opt_results)
    all_opt_results.append(opt_results)

all_opt_results = pd.concat(all_opt_results, axis=0)
all_opt_results = all_opt_results.groupby(all_opt_results.index).agg(
    ["mean", "std"])
all_opt_results = all_opt_results.sort_index(axis=1)
all_opt_results

# Visualize

In [ ]:
seed = 0
batch_n = 1

# Load
pth = "results/mfred/example-20230815-052223"

# Wind
# pth = "results/nrel/example-20230819-070441"

# 13, 7
# sample_choose = 93
rand_seed = np.random.randint(0,1000)
sample_choose = np.random.permutation(128)[10]

with open(f"{pth}-{seed}.pkl", "rb") as f:
    result = pickle.load(f)
    model_hyperparams = result["model_hyperparams"]
    data_params = result["data_params"]
    model_state_dict = result["Hierarchical NL"]["model_state_dict"]
from model import GeneralHNL
from dataset import generate_data_set, setup_seed

setup_seed(rand_seed)

(input_dim, output_dim, sample_rate, t, dltrain, dlval, dltest,
 input_timesteps, output_timesteps, train_mean, train_std,
 feature) = generate_data_set(**data_params)
m = GeneralHNL(**model_hyperparams).to(data_params["device"])
m.model.load_state_dict(model_state_dict)

train_mean, train_std = train_mean.cpu().numpy()[
    feature["fcst_feature"]], train_std.cpu().numpy()[feature["fcst_feature"]]
count = 0
for batch in dltest:
    if count == batch_n:
        break

    count += 1

labels = []
for avg_terms in [12, 3, 1]:
    data_to_predict = batch["data_to_predict"].transpose(1, 2)
    data_to_predict = torch.nn.functional.avg_pool1d(data_to_predict,
                                                     avg_terms, avg_terms)
    data_to_predict = data_to_predict.transpose(1, 2)
    labels.append(data_to_predict.detach().cpu().numpy())

all_decomp_fcsts, _ = m.model.decompose_predcit(batch["observed_data"],
                                                batch["available_forecasts"],
                                                batch["observed_tp"],
                                                batch["tp_to_predict"])

fig, axs = plt.subplots(3, 3, figsize=[15, 6])
fig2, axs2 = plt.subplots(3, figsize=[5, 6])
for i in range(len(all_decomp_fcsts)):
    sub_comps = all_decomp_fcsts[i]
    # print(len(sub_comps))
    fcsts = []
    for j in range(len(sub_comps)):
        comps = sub_comps[j]
        # print(comps)
        comps_plt = comps[sample_choose].squeeze(
        ) * train_std + train_mean if j == 0 else comps[sample_choose].squeeze(
        ) * train_std
        comps_plt = np.where(comps_plt > 16, 16, comps_plt)

        axs[i, j].plot(comps_plt, lw=2, c=plt.get_cmap("Set2").colors[j])
        axs[i, j].grid(alpha=0.5)
        axs[i, j].set_axisbelow(True)
        axs[i, j].set_axisbelow(True)
        if j + 1 >= 2:
            fcsts.append(comps[sample_choose].squeeze())
        else:
            fcsts.append(comps[sample_choose].squeeze())

    plt_fcst = np.sum(fcsts, axis=0) * train_std + train_mean
    plt_fcst = np.where(plt_fcst < 0, 0, plt_fcst)
    plt_fcst = np.where(plt_fcst > 16, 16, plt_fcst)
    axs2[i].plot(plt_fcst, lw=2, c="grey", label="predict")
    # axs2[i].plot(labels[i][sample_choose] * train_std + train_mean,
    #              lw=2,
    #              c="grey",
    #              ls="--",
    #              label="real")
    axs2[i].grid(alpha=0.5)

    axs2[i].set_axisbelow(True)

fig.tight_layout()
fig2.tight_layout()
fig.delaxes(axs[0, 1])
fig.delaxes(axs[0, 2])
fig.delaxes(axs[1, 2])

axs2[-1].set_xlabel("Assembled forecasts", fontweight="bold")
axs[-1, 0].set_xlabel("Component 1", fontweight="bold")
axs[-1, 1].set_xlabel("Component 2", fontweight="bold")
axs[-1, 2].set_xlabel("Component 3", fontweight="bold")
axs[0, 0].set_ylabel("60min", fontweight="bold")
axs[1, 0].set_ylabel("15min", fontweight="bold")
axs[2, 0].set_ylabel("5min", fontweight="bold")
axs2[0].set_ylabel("60min", fontweight="bold")
axs2[1].set_ylabel("15min", fontweight="bold")
axs2[2].set_ylabel("5min", fontweight="bold")
axs2[0].legend()